# 第14章: 時系列回帰と予測 (Time Series Regression and Forecasting)

この章では、時系列データの分析と予測について学びます。

## 主なトピック
- 時系列データの特性
- 自己回帰（AR）モデル
- 自己相関と偏自己相関
- 予測と予測評価

## 必要なパッケージ

In [ ]:
# 必要なパッケージのインストールと読み込み
if (!require("AER")) install.packages("AER")
if (!require("dynlm")) install.packages("dynlm")
if (!require("forecast")) install.packages("forecast")
if (!require("lmtest")) install.packages("lmtest")
if (!require("sandwich")) install.packages("sandwich")

library(AER)
library(dynlm)
library(forecast)
library(lmtest)
library(sandwich)

---
## 問題1: 時系列データの可視化と基本統計量

Rに組み込まれている `AirPassengers` データを使って、時系列データの基本的な特徴を確認します。

1. データをプロットして視覚的に確認してください
2. データの基本統計量（平均、分散、自己相関）を計算してください
3. トレンドと季節性があるかどうか判断してください

In [ ]:
# ここに回答を入力してください


### 模範解答

In [ ]:
# AirPassengersデータの読み込み
data(AirPassengers)

# 1. データのプロット
par(mfrow = c(2, 2))

# 時系列プロット
plot(AirPassengers, main = "航空旅客数の推移", 
     xlab = "年", ylab = "旅客数（千人）")

# 2. 自己相関関数（ACF）
acf(AirPassengers, main = "自己相関関数（ACF）")

# 偏自己相関関数（PACF）
pacf(AirPassengers, main = "偏自己相関関数（PACF）")

# 季節分解
decomp <- decompose(AirPassengers)
plot(decomp)

par(mfrow = c(1, 1))

# 基本統計量
cat("基本統計量:\n")
cat("平均:", mean(AirPassengers), "\n")
cat("標準偏差:", sd(AirPassengers), "\n")
cat("最小値:", min(AirPassengers), "\n")
cat("最大値:", max(AirPassengers), "\n")

# 3. トレンドと季節性の判断
cat("\n観察結果:\n")
cat("- 明確な上昇トレンドが存在します\n")
cat("- 12ヶ月周期の季節性が存在します\n")
cat("- 分散が時間とともに増加しています（分散不均一）\n")

---
## 問題2: AR(1)モデルの推定

シミュレーションでAR(1)過程を生成し、パラメータを推定します。

```r
set.seed(123)
n <- 200
phi <- 0.7  # 自己回帰係数
y <- numeric(n)
y[1] <- rnorm(1)
for (t in 2:n) {
  y[t] <- phi * y[t-1] + rnorm(1)
}
ar1_data <- ts(y)
```

1. 生成したデータをプロットしてください
2. AR(1)モデルを推定し、係数を確認してください
3. 推定値と真の値（0.7）を比較してください

In [ ]:
# ここに回答を入力してください


### 模範解答

In [ ]:
# AR(1)過程の生成
set.seed(123)
n <- 200
phi <- 0.7
y <- numeric(n)
y[1] <- rnorm(1)
for (t in 2:n) {
  y[t] <- phi * y[t-1] + rnorm(1)
}
ar1_data <- ts(y)

# 1. データのプロット
par(mfrow = c(1, 3))
plot(ar1_data, main = "AR(1)過程", xlab = "時間", ylab = "y")
acf(ar1_data, main = "ACF")
pacf(ar1_data, main = "PACF")
par(mfrow = c(1, 1))

# 2. AR(1)モデルの推定
# 方法1: ar()関数
ar1_fit <- ar(ar1_data, order.max = 1, method = "ols")
cat("ar()関数による推定:\n")
print(ar1_fit)

# 方法2: dynlm()関数
ar1_lm <- dynlm(ar1_data ~ L(ar1_data, 1))
cat("\ndynlm()関数による推定:\n")
summary(ar1_lm)

# 3. 推定値と真の値の比較
cat("\n比較:\n")
cat("真の係数: 0.7\n")
cat("ar()推定値:", ar1_fit$ar, "\n")
cat("dynlm()推定値:", coef(ar1_lm)[2], "\n")

---
## 問題3: AR(p)モデルの次数選択

AR(2)過程を生成し、情報量基準を使って次数を選択します。

```r
set.seed(456)
n <- 300
phi1 <- 0.5
phi2 <- 0.3
y <- numeric(n)
y[1:2] <- rnorm(2)
for (t in 3:n) {
  y[t] <- phi1 * y[t-1] + phi2 * y[t-2] + rnorm(1)
}
ar2_data <- ts(y)
```

1. PACFを見て次数を判断してください
2. AICとBICを使って最適次数を選択してください
3. 選択されたモデルを推定してください

In [ ]:
# ここに回答を入力してください


### 模範解答

In [ ]:
# AR(2)過程の生成
set.seed(456)
n <- 300
phi1 <- 0.5
phi2 <- 0.3
y <- numeric(n)
y[1:2] <- rnorm(2)
for (t in 3:n) {
  y[t] <- phi1 * y[t-1] + phi2 * y[t-2] + rnorm(1)
}
ar2_data <- ts(y)

# 1. PACFによる次数判断
par(mfrow = c(1, 2))
acf(ar2_data, main = "ACF")
pacf(ar2_data, main = "PACF")
par(mfrow = c(1, 1))

cat("PACFは2次までで有意であり、それ以降は急激に減少しています。\n")
cat("これはAR(2)過程の特徴です。\n\n")

# 2. AICとBICによる次数選択
aic_values <- numeric(6)
bic_values <- numeric(6)

for (p in 1:6) {
  model <- ar(ar2_data, order.max = p, aic = FALSE, method = "ols")
  # 手動でAIC/BICを計算
  residuals <- na.omit(ar2_data - fitted(arima(ar2_data, order = c(p, 0, 0))))
  n_eff <- length(residuals)
  sigma2 <- sum(residuals^2) / n_eff
  aic_values[p] <- n_eff * log(sigma2) + 2 * (p + 1)
  bic_values[p] <- n_eff * log(sigma2) + log(n_eff) * (p + 1)
}

# ar()関数の自動選択
ar_aic <- ar(ar2_data, method = "ols")
cat("ar()関数によるAIC最小次数:", ar_aic$order, "\n")

# 情報量基準の比較
cat("\n次数ごとの情報量基準:\n")
print(data.frame(p = 1:6, AIC = round(aic_values, 2), BIC = round(bic_values, 2)))

# 3. 選択されたモデルの推定
ar2_fit <- arima(ar2_data, order = c(2, 0, 0))
cat("\nAR(2)モデルの推定結果:\n")
print(ar2_fit)

cat("\n真の係数との比較:\n")
cat("phi1: 真値 = 0.5, 推定値 =", ar2_fit$coef[1], "\n")
cat("phi2: 真値 = 0.3, 推定値 =", ar2_fit$coef[2], "\n")

---
## 問題4: 分布ラグモデル

説明変数の過去の値も含めたADL（Autoregressive Distributed Lag）モデルを推定します。

```r
set.seed(789)
n <- 150
x <- cumsum(rnorm(n))  # ランダムウォーク
# y は x の現在と1期ラグに依存
y <- numeric(n)
y[1] <- rnorm(1)
for (t in 2:n) {
  y[t] <- 0.3 * y[t-1] + 0.5 * x[t] + 0.2 * x[t-1] + rnorm(1, 0, 0.5)
}
adl_data <- ts(cbind(y = y, x = x))
```

1. ADL(1,1)モデルを推定してください
2. 短期乗数と長期乗数を計算してください
3. 累積インパルス応答を計算してください

In [ ]:
# ここに回答を入力してください


### 模範解答

In [ ]:
# データの生成
set.seed(789)
n <- 150
x <- cumsum(rnorm(n))
y <- numeric(n)
y[1] <- rnorm(1)
for (t in 2:n) {
  y[t] <- 0.3 * y[t-1] + 0.5 * x[t] + 0.2 * x[t-1] + rnorm(1, 0, 0.5)
}
adl_data <- ts(cbind(y = y, x = x))

# 1. ADL(1,1)モデルの推定
adl_model <- dynlm(y ~ L(y, 1) + x + L(x, 1), data = adl_data)
cat("ADL(1,1)モデルの推定結果:\n")
summary(adl_model)

# 係数の抽出
beta_y1 <- coef(adl_model)["L(y, 1)"]
beta_x0 <- coef(adl_model)["x"]
beta_x1 <- coef(adl_model)["L(x, 1)"]

# 2. 短期乗数と長期乗数
short_run <- beta_x0  # 短期乗数（衝撃乗数）
long_run <- (beta_x0 + beta_x1) / (1 - beta_y1)  # 長期乗数

cat("\n乗数の計算:\n")
cat("短期乗数（衝撃乗数）:", short_run, "\n")
cat("長期乗数:", long_run, "\n")
cat("\n真の値:\n")
cat("短期乗数: 0.5\n")
cat("長期乗数: (0.5 + 0.2) / (1 - 0.3) = 1\n")

# 3. 累積インパルス応答
horizons <- 10
impulse_response <- numeric(horizons)
impulse_response[1] <- beta_x0

for (h in 2:horizons) {
  if (h == 2) {
    impulse_response[h] <- beta_y1 * impulse_response[1] + beta_x1
  } else {
    impulse_response[h] <- beta_y1 * impulse_response[h-1]
  }
}

cumulative_response <- cumsum(impulse_response)

cat("\n累積インパルス応答:\n")
print(data.frame(期間 = 1:horizons, 
                 インパルス応答 = round(impulse_response, 4),
                 累積応答 = round(cumulative_response, 4)))

# グラフ
par(mfrow = c(1, 2))
barplot(impulse_response, names.arg = 1:horizons, 
        main = "インパルス応答", xlab = "期間", ylab = "応答")
plot(1:horizons, cumulative_response, type = "b", pch = 16,
     main = "累積インパルス応答", xlab = "期間", ylab = "累積応答")
abline(h = long_run, lty = 2, col = "red")
par(mfrow = c(1, 1))

---
## 問題5: 時系列の予測

AR(1)モデルを使って予測を行い、予測精度を評価します。

```r
set.seed(111)
n <- 200
y <- arima.sim(n = n, model = list(ar = 0.8), sd = 1)
y_ts <- ts(y)
```

1. データを訓練セット（最初の180期）とテストセット（残り20期）に分割してください
2. 訓練セットでAR(1)モデルを推定し、テスト期間を予測してください
3. 予測誤差（RMSE, MAE）を計算してください

In [ ]:
# ここに回答を入力してください


### 模範解答

In [ ]:
# データの生成
set.seed(111)
n <- 200
y <- arima.sim(n = n, model = list(ar = 0.8), sd = 1)
y_ts <- ts(y)

# 1. データの分割
train <- window(y_ts, end = 180)
test <- window(y_ts, start = 181)

cat("訓練セットのサイズ:", length(train), "\n")
cat("テストセットのサイズ:", length(test), "\n\n")

# 2. モデル推定と予測
ar1_model <- arima(train, order = c(1, 0, 0))
cat("推定されたAR(1)係数:", ar1_model$coef[1], "\n\n")

# 予測
forecasts <- forecast(ar1_model, h = 20)

# 3. 予測精度の評価
actual <- as.numeric(test)
predicted <- as.numeric(forecasts$mean)

# 予測誤差の計算
errors <- actual - predicted
rmse <- sqrt(mean(errors^2))
mae <- mean(abs(errors))
mape <- mean(abs(errors / actual)) * 100

cat("予測精度の評価:\n")
cat("RMSE:", rmse, "\n")
cat("MAE:", mae, "\n")
cat("MAPE:", mape, "%\n")

# グラフ
plot(forecasts, main = "AR(1)モデルによる予測")
lines(181:200, actual, col = "red", type = "b", pch = 16)
legend("topleft", legend = c("予測", "実績"), col = c("blue", "red"), lty = 1, pch = c(NA, 16))

---
## 問題6: 系列相関の検定

回帰モデルの残差に系列相関があるかどうかを検定します。

```r
set.seed(222)
n <- 100
x <- rnorm(n)
# 誤差項にAR(1)過程を持つ
e <- arima.sim(n = n, model = list(ar = 0.6), sd = 1)
y <- 2 + 3 * x + e
serial_data <- data.frame(y = y, x = x)
```

1. 通常のOLS回帰を行ってください
2. Durbin-Watson検定を実施してください
3. Breusch-Godfrey検定を実施してください

In [ ]:
# ここに回答を入力してください


### 模範解答

In [ ]:
# データの生成
set.seed(222)
n <- 100
x <- rnorm(n)
e <- arima.sim(n = n, model = list(ar = 0.6), sd = 1)
y <- 2 + 3 * x + e
serial_data <- data.frame(y = y, x = x)

# 1. OLS回帰
ols_model <- lm(y ~ x, data = serial_data)
cat("OLS回帰の結果:\n")
summary(ols_model)

# 残差のプロット
par(mfrow = c(1, 2))
plot(residuals(ols_model), type = "l", main = "残差の時系列プロット")
acf(residuals(ols_model), main = "残差のACF")
par(mfrow = c(1, 1))

# 2. Durbin-Watson検定
cat("\nDurbin-Watson検定:\n")
dw_test <- dwtest(ols_model)
print(dw_test)

cat("\n解釈:\n")
cat("DW統計量が2より大きく離れている場合、系列相関の存在が示唆されます。\n")
cat("DW < 2: 正の系列相関, DW > 2: 負の系列相関\n\n")

# 3. Breusch-Godfrey検定
cat("Breusch-Godfrey検定（ラグ1）:\n")
bg_test1 <- bgtest(ols_model, order = 1)
print(bg_test1)

cat("\nBreusch-Godfrey検定（ラグ4）:\n")
bg_test4 <- bgtest(ols_model, order = 4)
print(bg_test4)

cat("\n結論:\n")
cat("p値が小さい場合、帰無仮説（系列相関なし）を棄却し、\n")
cat("系列相関の存在が示唆されます。\n")

---
## 問題7: Newey-West標準誤差

系列相関がある場合、HAC（Heteroskedasticity and Autocorrelation Consistent）標準誤差を使用します。

問題6のデータを使って：

1. 通常の標準誤差とNewey-West標準誤差を比較してください
2. 両方の標準誤差を使った信頼区間を計算してください
3. 推論にどのような影響があるか考察してください

In [ ]:
# ここに回答を入力してください


### 模範解答

In [ ]:
# 問題6のデータを再生成
set.seed(222)
n <- 100
x <- rnorm(n)
e <- arima.sim(n = n, model = list(ar = 0.6), sd = 1)
y <- 2 + 3 * x + e
serial_data <- data.frame(y = y, x = x)
ols_model <- lm(y ~ x, data = serial_data)

# 1. 標準誤差の比較
cat("通常の標準誤差:\n")
print(coeftest(ols_model))

cat("\nNewey-West標準誤差:\n")
nw_se <- coeftest(ols_model, vcov = NeweyWest(ols_model))
print(nw_se)

# 2. 信頼区間の比較
cat("\n95%信頼区間の比較（x の係数）:\n")

# 通常の信頼区間
ci_ols <- confint(ols_model)["x", ]
cat("通常のSE:", ci_ols, "\n")

# Newey-West信頼区間
coef_x <- coef(ols_model)["x"]
se_nw <- nw_se["x", "Std. Error"]
ci_nw <- c(coef_x - 1.96 * se_nw, coef_x + 1.96 * se_nw)
cat("Newey-West SE:", ci_nw, "\n")

# 3. 標準誤差の比較表
comparison <- data.frame(
  変数 = c("(Intercept)", "x"),
  通常SE = summary(ols_model)$coefficients[, "Std. Error"],
  NeweyWestSE = nw_se[, "Std. Error"]
)
comparison$比率 <- comparison$NeweyWestSE / comparison$通常SE

cat("\n標準誤差の比較:\n")
print(comparison)

cat("\n考察:\n")
cat("- 正の系列相関がある場合、通常の標準誤差は過小推定される傾向があります\n")
cat("- Newey-West標準誤差は系列相関を考慮するため、より保守的（大きい）になります\n")
cat("- これにより、t統計量が小さくなり、統計的有意性の判断が変わる可能性があります\n")

---
## 問題8: 定常性とランダムウォーク

定常過程と非定常過程（ランダムウォーク）の違いを理解します。

```r
set.seed(333)
n <- 200

# 定常AR(1)過程
stationary <- arima.sim(n = n, model = list(ar = 0.7), sd = 1)

# ランダムウォーク（非定常）
random_walk <- cumsum(rnorm(n))
```

1. 両方の過程をプロットして比較してください
2. ACFの形状を比較してください
3. ランダムウォークの1階差分を取り、定常性を確認してください

In [ ]:
# ここに回答を入力してください


### 模範解答

In [ ]:
# データの生成
set.seed(333)
n <- 200
stationary <- arima.sim(n = n, model = list(ar = 0.7), sd = 1)
random_walk <- cumsum(rnorm(n))

# 1. プロットの比較
par(mfrow = c(2, 2))

plot(stationary, main = "定常AR(1)過程", ylab = "y")
abline(h = 0, lty = 2, col = "red")

plot(random_walk, main = "ランダムウォーク（非定常）", ylab = "y", type = "l")

# 2. ACFの比較
acf(stationary, main = "定常AR(1)のACF")
acf(random_walk, main = "ランダムウォークのACF")

par(mfrow = c(1, 1))

cat("観察結果:\n")
cat("- 定常過程: 平均周りを変動し、ACFは指数的に減衰\n")
cat("- ランダムウォーク: トレンドがあり、ACFは非常にゆっくり減衰\n\n")

# 3. 1階差分
rw_diff <- diff(random_walk)

par(mfrow = c(1, 2))
plot(rw_diff, main = "ランダムウォークの1階差分", ylab = "差分")
abline(h = 0, lty = 2, col = "red")
acf(rw_diff, main = "1階差分のACF")
par(mfrow = c(1, 1))

cat("\n1階差分の結果:\n")
cat("平均:", mean(rw_diff), "\n")
cat("分散:", var(rw_diff), "\n")
cat("\n1階差分を取ることで、ランダムウォークはホワイトノイズ（定常過程）になります。\n")

---
## 問題9: 見せかけの回帰

2つの独立なランダムウォークを回帰すると、高い決定係数が得られる「見せかけの回帰」現象を確認します。

```r
set.seed(444)
n <- 200
# 2つの独立なランダムウォーク
y <- cumsum(rnorm(n))
x <- cumsum(rnorm(n))
spurious_data <- data.frame(y = y, x = x)
```

1. yをxに回帰し、結果を確認してください
2. R²とDW統計量を確認してください
3. 1階差分を使って再推定してください

In [ ]:
# ここに回答を入力してください


### 模範解答

In [ ]:
# データの生成
set.seed(444)
n <- 200
y <- cumsum(rnorm(n))
x <- cumsum(rnorm(n))
spurious_data <- data.frame(y = y, x = x)

# 1. 見せかけの回帰
spurious_reg <- lm(y ~ x, data = spurious_data)
cat("見せかけの回帰の結果:\n")
summary(spurious_reg)

# 2. R²とDW統計量
cat("\nR²:", summary(spurious_reg)$r.squared, "\n")
cat("DW統計量:", dwtest(spurious_reg)$statistic, "\n")

cat("\n見せかけの回帰の特徴:\n")
cat("- 高いR²（実際には無関係なのに）\n")
cat("- DW統計量が0に近い（強い正の系列相関）\n")
cat("- t統計量が有意（偽の関係）\n\n")

# グラフ
par(mfrow = c(1, 2))
plot(x, y, main = "見せかけの関係", pch = 16, cex = 0.5)
abline(spurious_reg, col = "red")
plot(residuals(spurious_reg), type = "l", main = "残差の推移")
par(mfrow = c(1, 1))

# 3. 1階差分による推定
dy <- diff(y)
dx <- diff(x)
diff_reg <- lm(dy ~ dx)

cat("1階差分を使った回帰:\n")
summary(diff_reg)

cat("\n差分モデルのR²:", summary(diff_reg)$r.squared, "\n")
cat("差分モデルのDW統計量:", dwtest(diff_reg)$statistic, "\n")

cat("\n結論:\n")
cat("差分を取ることで、見せかけの関係が消え、\n")
cat("R²は0に近くなり、真の（無）関係が明らかになります。\n")

---
## 問題10: GDPデータを使った実践

USMacroG（米国マクロ経済データ）を使って、実データでの時系列分析を行います。

1. 実質GDPの対数系列を作成し、トレンドを確認してください
2. 対数差分（成長率）を計算し、その時系列特性を確認してください
3. GDP成長率のAR(1)モデルを推定し、1期先予測を行ってください

In [ ]:
# ここに回答を入力してください


### 模範解答

In [ ]:
# USMacroGデータの読み込み
data("USMacroG")

# 1. 実質GDPの対数系列
log_gdp <- log(USMacroG[, "gdp"])

par(mfrow = c(2, 2))
plot(USMacroG[, "gdp"], main = "実質GDP", ylab = "GDP")
plot(log_gdp, main = "対数GDP", ylab = "log(GDP)")

# 2. 対数差分（成長率）
gdp_growth <- diff(log_gdp) * 100  # パーセント表示

plot(gdp_growth, main = "GDP成長率（四半期）", ylab = "%")
abline(h = 0, lty = 2, col = "red")
abline(h = mean(gdp_growth), lty = 2, col = "blue")

acf(gdp_growth, main = "成長率のACF")
par(mfrow = c(1, 1))

cat("GDP成長率の基本統計量:\n")
cat("平均:", mean(gdp_growth), "%\n")
cat("標準偏差:", sd(gdp_growth), "%\n")
cat("最小:", min(gdp_growth), "%\n")
cat("最大:", max(gdp_growth), "%\n\n")

# 3. AR(1)モデルの推定と予測
ar1_gdp <- arima(gdp_growth, order = c(1, 0, 0))
cat("AR(1)モデルの推定結果:\n")
print(ar1_gdp)

# 1期先予測
forecast_1 <- predict(ar1_gdp, n.ahead = 1)
cat("\n1期先予測:\n")
cat("予測値:", forecast_1$pred, "%\n")
cat("95%予測区間:", 
    forecast_1$pred - 1.96 * forecast_1$se, "から",
    forecast_1$pred + 1.96 * forecast_1$se, "%\n")

# 複数期先予測
forecast_multi <- forecast(ar1_gdp, h = 8)
plot(forecast_multi, main = "GDP成長率の予測")

---
## まとめ

この章では以下の内容を学びました：

1. **時系列データの特性**: トレンド、季節性、自己相関
2. **AR(p)モデル**: 自己回帰モデルの推定と次数選択
3. **分布ラグモデル**: 短期乗数と長期乗数の計算
4. **予測**: モデルに基づく予測と予測精度の評価
5. **系列相関の検定**: Durbin-Watson検定、Breusch-Godfrey検定
6. **HAC標準誤差**: Newey-West標準誤差による頑健な推論
7. **定常性**: 定常過程とランダムウォークの違い
8. **見せかけの回帰**: 非定常データの回帰における問題